In [1]:
import json
import pathlib
import shutil

import cv2
from tqdm.notebook import tqdm

YOLO_DIR = pathlib.Path('../data/yolo')
YOLO_IMAGE_DIRNAME = 'images'
YOLO_LABEL_DIRNAME = 'labels'
YOLO_TRAIN_DIRNAME = 'train'
YOLO_VALID_DIRNAME = 'valid'

TRAIN_IMAGE_DIR = pathlib.Path('../data/train_bbox_images')
TRAIN_ANNOTATIONS = pathlib.Path('../data/train_bbox_annotations.json')
TEST_IMAGE_DIR = pathlib.Path('../data/evaluation_bbox_images')
CV_SPLIT_CONFIG = pathlib.Path('../data/cv.json')

In [2]:
with open(TRAIN_ANNOTATIONS, 'r') as f:
    annotations = json.load(f)['images']
    
annotations

[{'file_name': 'train_0.tif',
  'width': 500,
  'height': 375,
  'annotations': [{'class': 'vacant_lot',
    'bbox': [311.0, 36.246, 35.0, 45.145999999999994]},
   {'class': 'vacant_lot', 'bbox': [121.0, 156.0, 203.0, 144.0]}]},
 {'file_name': 'train_1.tif',
  'width': 500,
  'height': 375,
  'annotations': [{'class': 'vacant_lot',
    'bbox': [198.0, 231.0, 56.0, 34.0]}]},
 {'file_name': 'train_10.tif',
  'width': 500,
  'height': 375,
  'annotations': [{'class': 'vacant_lot',
    'bbox': [440.0, 150.0, 36.0, 44.0]}]},
 {'file_name': 'train_100.tif',
  'width': 500,
  'height': 375,
  'annotations': [{'class': 'vacant_lot',
    'bbox': [385.918, 24.482, 60.488, 41.468999999999994]},
   {'class': 'vacant_lot',
    'bbox': [403.946, 88.866, 43.77299999999997, 36.973]},
   {'class': 'vacant_lot',
    'bbox': [426.423, 143.236, 43.53199999999998, 32.372000000000014]},
   {'class': 'vacant_lot',
    'bbox': [315.417, 248.482, 39.636000000000024, 44.80600000000001]}]},
 {'file_name': 'train

In [3]:
def bbox_to_yolo_format(bbox, image_width, image_height):
    cls_no = 0
    left, top, w, h = bbox

    x = left + w / 2 # Center x
    y = top + h / 2 # Center y
    
    x /= image_width
    y /= image_height
    w /= image_width
    h /= image_height
    
    return f'{cls_no} {x} {y} {w} {h}\n'


def bbox_to_mityolo_format(bbox, image_width, image_height):
    cls_no = 0
    left, top, w, h = bbox

    right = left + w
    bottom = top + h
    
    left /= image_width
    top /= image_height
    right /= image_width
    bottom /= image_height
    
    return f'{cls_no} {left} {top} {right} {bottom}\n'



with open(CV_SPLIT_CONFIG, 'r') as f:
    folds = json.load(f)
    
    
for fold, indices in folds.items():
    train_image_dir = YOLO_DIR / fold / YOLO_IMAGE_DIRNAME / YOLO_TRAIN_DIRNAME
    train_label_dir = YOLO_DIR / fold / YOLO_LABEL_DIRNAME / YOLO_TRAIN_DIRNAME
    
    train_image_dir.mkdir(parents=True, exist_ok=True)
    train_label_dir.mkdir(parents=True, exist_ok=True)
    
    for idx in tqdm(indices['train'], desc=f'Processing fold {fold} (train)'):
        imgfile = annotations[idx]['file_name']
        image_width = annotations[idx]['width']
        image_height = annotations[idx]['height']
        # shutil.copy(TRAIN_IMAGE_DIR / imgfile, train_image_dir / imgfile)
        # img = cv2.imread(str(TRAIN_IMAGE_DIR / imgfile))
        # cv2.imwrite(str((train_image_dir / imgfile).with_suffix('.png')), img)
        
        anno_file = train_label_dir / imgfile.replace('.tif', '.txt')
        with open(anno_file, 'w') as anno_f:
            for bbox in annotations[idx]['annotations']:
                # anno_f.write(bbox_to_yolo_format(bbox['bbox'], image_width, image_height))
                anno_f.write(bbox_to_mityolo_format(bbox['bbox'], image_width, image_height))
    
    valid_image_dir = YOLO_DIR / fold / YOLO_IMAGE_DIRNAME / YOLO_VALID_DIRNAME
    valid_label_dir = YOLO_DIR / fold / YOLO_LABEL_DIRNAME / YOLO_VALID_DIRNAME
    
    valid_image_dir.mkdir(parents=True, exist_ok=True)
    valid_label_dir.mkdir(parents=True, exist_ok=True)
    
    for idx in tqdm(indices['valid'], desc=f'Processing fold {fold} (valid)'):
        imgfile = annotations[idx]['file_name']
        image_width = annotations[idx]['width']
        image_height = annotations[idx]['height']
        # shutil.copy(TRAIN_IMAGE_DIR / imgfile, valid_image_dir / imgfile)
        # img = cv2.imread(str(TRAIN_IMAGE_DIR / imgfile))
        # cv2.imwrite(str((valid_image_dir / imgfile).with_suffix('.png')), img)
        
        anno_file = valid_label_dir / imgfile.replace('.tif', '.txt')
        with open(anno_file, 'w') as anno_f:
            for bbox in annotations[idx]['annotations']:
                # anno_f.write(bbox_to_yolo_format(bbox['bbox'], image_width, image_height))
                anno_f.write(bbox_to_mityolo_format(bbox['bbox'], image_width, image_height))
                

Processing fold 0 (train):   0%|          | 0/480 [00:00<?, ?it/s]

Processing fold 0 (valid):   0%|          | 0/120 [00:00<?, ?it/s]

Processing fold 1 (train):   0%|          | 0/480 [00:00<?, ?it/s]

Processing fold 1 (valid):   0%|          | 0/120 [00:00<?, ?it/s]

Processing fold 2 (train):   0%|          | 0/480 [00:00<?, ?it/s]

Processing fold 2 (valid):   0%|          | 0/120 [00:00<?, ?it/s]

Processing fold 3 (train):   0%|          | 0/480 [00:00<?, ?it/s]

Processing fold 3 (valid):   0%|          | 0/120 [00:00<?, ?it/s]

Processing fold 4 (train):   0%|          | 0/480 [00:00<?, ?it/s]

Processing fold 4 (valid):   0%|          | 0/120 [00:00<?, ?it/s]

In [4]:
test_imgfiles = sorted(TEST_IMAGE_DIR.glob('*.tif'))
outdir = YOLO_DIR / 'evaluation/images'
outdir.mkdir(parents=True, exist_ok=True)

for imgfile in tqdm(test_imgfiles, desc='Processing test images'):
    img = cv2.imread(str(imgfile))
    
    cv2.imwrite(str(outdir /  f'{imgfile.stem}.png'), img)

Processing test images:   0%|          | 0/400 [00:00<?, ?it/s]